In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
np.random.seed(151)

In [3]:
import chainer.links as L
import chainer.functions as F

In [4]:
from chainer import Chain

In [5]:
from chainer import Variable
from chainer.optimizers import SGD
from chainer.datasets import TupleDataset
from chainer.iterators import SerialIterator
from chainer.training import StandardUpdater
from chainer.training import Trainer

# Deep Learningドリル - アクティベーション関数編 -

ここまででみなさんはchainerをある程度扱えるようになってきたと思います。いよいよ、Deep Learningにおける個別の様々な性質について学びます。これから学ぶことは基本的にchainerには依存しないことですので、Tensorflowなど別のDeep Learningフレームワークを使う場合でも役に立つはずです。

## Softmax関数の性質

まずはsoftmax関数について学びましょう。softmax関数は今まで何度も使ってきましたね。
softmax関数の性質について、覚えて頂きたいことが3つあります。
初めに列挙します。


1. 出力値は$[0, 1]$の範囲に収まり、各次元の和を計算すると$1$になる
2. ベクトルの各要素に定数を加算してもsoftmaxの結果は変わらない($\forall b \in R, x_i \in R, \mathrm{softmax}(x_1, x_2, \ldots, x_n) = \mathrm{softmax}(x_1+b, x_2+b, \ldots, x_n+b)$)
3. Softmaxの出力が2次元のとき、Sigmoid関数と一致する

### 出力値は$[0, 1]$の範囲に収まり、各次元の和を計算すると$1$になる

これは簡単ですね。softmaxの式は入力の各次元を指数乗した上で、全体の和で割っています。割る数は割られる数より必ず大きくなりますので、結果は$1$以下となります。また、softmaxは各次元の和が$1$になるように定義されています。この性質により、 __softmax関数の出力はカテゴリカル変数に対する確率分布である__ と解釈することができます。

なお、当然ですが1次元ベクトルを入力すると出力は必ず1になります。

In [10]:
x = np.random.random(1).astype(np.float32).reshape((1,1))

In [11]:
x

array([[ 0.31071031]], dtype=float32)

In [12]:
F.softmax(Variable(x))

variable([[ 1.]])

### ベクトルの各要素に定数を加算しても、softmaxの結果は変わらない

これは試しに実験してみましょう。各要素の値がちょうど1ずつ違うベクトルを2つ用意します。

In [13]:
x1 = np.random.random(5).astype(np.float32).reshape((1,5))
x2 = x1 + 1

くっつけます

In [14]:
x = np.r_[x1, x2]

In [15]:
x

array([[ 0.80031389,  0.57449448,  0.76997077,  0.9507612 ,  0.75278902],
       [ 1.80031395,  1.57449448,  1.76997077,  1.9507612 ,  1.75278902]], dtype=float32)

1行目と2行目でちょうど1ずつ違いますね。これをsoftmax関数にかけてみましょう。

In [16]:
F.softmax(Variable(x))

variable([[ 0.20475373,  0.16336526,  0.19863418,  0.23799632,  0.19525044],
          [ 0.20475374,  0.16336526,  0.19863418,  0.23799632,  0.19525044]])

結果がほぼ同じになりましたね。

### Softmaxの出力が2次元のとき、Sigmoid関数と一致する

これもやってみましょう

In [19]:
x = np.random.random(2).astype(np.float32).reshape((1,2))

In [20]:
x

array([[ 0.44673532,  0.52645624]], dtype=float32)

まずはxに対するsoftmaxを計算しましょう。

In [21]:
F.softmax(Variable(x))

variable([[ 0.48008034,  0.51991969]])

続いて、sigmoidで同じ結果を得るには以下のように計算します。

In [22]:
z = (x - x[0, 0])[0, 1:]
F.sigmoid(Variable(z))

variable([ 0.51991969])

In [35]:
z

array([ 0.07972091], dtype=float32)

2次元目が同じになりましたね。この2つの式に対するlossの値は一致します。正解ラベルが0のときと1のとき両方どうなるか見てみましょう。

In [24]:
F.softmax_cross_entropy(Variable(x), Variable(np.array([1], dtype=np.int32)), normalize=False, reduce="no")

variable([ 0.65408099])

In [25]:
F.sigmoid_cross_entropy(Variable(z), Variable(np.array([1], dtype=np.int32)), normalize=False, reduce="no")

variable([ 0.65408099])

In [26]:
F.softmax_cross_entropy(Variable(x), Variable(np.array([0], dtype=np.int32)), normalize=False, reduce="no")

variable([ 0.7338019])

In [27]:
F.sigmoid_cross_entropy(Variable(z), Variable(np.array([0], dtype=np.int32)), normalize=False, reduce="no")

variable([ 0.7338019])

In [30]:
Variable(np.array([1], dtype=np.int32))

variable([1])

このようにsoftmaxの出力が2次元のとき、完全に同一なネットワークをsigmoidを使って構成することができます。